In [4]:
# -*- coding: utf-8 -*-
"""
Created on Wed May 29 16:22:12 2019

@author: asus
"""

# suggested by mdaoust
# from https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset

import keras
from keras.datasets import imdb

NUM_WORDS = 5000 # only use top 5000 words
INDEX_FROM = 3   # word index offset

(train_x, train_y), (test_x, test_y) = keras.datasets.imdb.load_data(
        num_words=NUM_WORDS, index_from=INDEX_FROM)

word_to_id = imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
id_to_word = {value:key for key, value in word_to_id.items()}

review_id = 25
print(' '.join(id_to_word[num] for num in train_x[review_id] ))




<START> this is a very light headed comedy about a wonderful family that has a son called <UNK> because he use to <UNK> at his food <UNK> loves to take all kinds of pictures of the people in a small <UNK> of <UNK> <UNK> and manages to get the attention of a group of photo art lovers from new york city <UNK> has a cute sister who goes simply nuts over <UNK> and is actually an <UNK> taking <UNK> of <UNK> from a bag there are scenes of men showing off the <UNK> in their <UNK> with <UNK> movements and <UNK> doing pretty much the same it is rather hard to keep your mind out of the <UNK> with this film but who cares it is only a film to give you a few laughs at a simple picture made in <UNK>


In [ ]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

np.random.seed(42)

# Максимальное кол-во слов (по частоте использования)
max_features = 5000

# Загружаем данные
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# Максимальная длина рецензии в словах
maxlen = 90

# Заполняем или обрезаем рецензии
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

# Создаем сеть
model = Sequential()
# Слой векторного представления слов
model.add(Embedding(max_features, 48))
model.add(keras.layers.SpatialDropout1D(0.2))
# Слой долго-краткосрочной памяти
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
# Полносвязный слой для классификации
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test), verbose=1)

# Проверяем качество обучения на тестовых данных
scores = model.evaluate(X_test, y_test, batch_size=32)

print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))


# Сохраняем нейронную сеть

# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("LSTM.json", "w")
json_file.write(model_json)
json_file.close()

model.save_weights("LSTM.h5")

print ("Сохранили Model")

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 71s 3ms/step - loss: 0.4858 - acc: 0.7653 - val_loss: 0.4187 - val_acc: 0.8096
Epoch 2/10
25000/25000 [==============================] - 70s 3ms/step - loss: 0.3676 - acc: 0.8446 - val_loss: 0.3944 - val_acc: 0.8207
Epoch 3/10
25000/25000 [==============================] - 74s 3ms/step - loss: 0.3306 - acc: 0.8631 - val_loss: 0.3652 - val_acc: 0.8373
Epoch 4/10
25000/25000 [==============================] - 70s 3ms/step - loss: 0.2997 - acc: 0.8764 - val_loss: 0.3655 - val_acc: 0.8361
Epoch 5/10
25000/25000 [==============================] - 65s 3ms/step - loss: 0.2824 - acc: 0.8854 - val_loss: 0.3808 - val_acc: 0.8350
Epoch 6/10
25000/25000 [==============================] - 65s 3ms/step - loss: 0.2566 - acc: 0.8968 - val_loss: 0.3998 - val_acc: 0.8392
Epoch 7/10
25000/25000 [==============================] - 65s 3ms/step - loss: 0.2371 - acc: 0.9046 - val_loss: 0.3801 - val_acc

In [7]:
import numpy as np
from keras.preprocessing import sequence
from keras.preprocessing.text import hashing_trick
from keras.models import model_from_json
json_file = open("LSTM.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("LSTM.h5")
#  Компилируем модель
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#  Загружаем рецензию для распознавания
f=open("review.txt", "r")
text = f.read()
f.close()

x = hashing_trick(text, 5000, hash_function=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')
x = np.array(x)
x = np.expand_dims(x, axis=0)
x = sequence.pad_sequences(x, maxlen = 90)

#  Запускаем распознавание

prediction = loaded_model.predict(x)

print(prediction)

[[0.91412]]


In [8]:
f=open("review-Copy1.txt", "r")
text = f.read()
f.close()

x = hashing_trick(text, 5000, hash_function=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')
x = np.array(x)
x = np.expand_dims(x, axis=0)
x = sequence.pad_sequences(x, maxlen = 90)

#  Запускаем распознавание

prediction = loaded_model.predict(x)

print(prediction)

[[0.0346388]]


In [ ]:
np.concatenate()